# Форматы данных (2)

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

In [ ]:
import csv
 
with open('/content/drive/MyDrive/Colab Notebooks/lab 5/open_pubs.csv') as fp:
    reader = csv.reader(fp)
    header = next(reader)
    
    for row in reader:
        print(row)
        break

['24', 'Anchor Inn', 'Upper Street, Stratford St Mary, COLCHESTER, Essex', 'CO7 6LW', '604748', '234405', '51.97039', '0.979328', 'Babergh']


2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [ ]:
import numpy as np
 
A = np.random.randint(0, 100, size=(10_000, 10_000))
B = np.random.randint(0, 100, size=(10_000, 10_000))
 
np.save('A.npy', A)
np.savez('AB.npz', A, B)
 
r = np.load('AB.npz')
print(r.files)
 
r['arr_0']
 
A.dtype
 
10_000 * 10_000 * 4 / 1024 / 1024

['arr_0', 'arr_1']


381.4697265625

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

In [ ]:
import h5py
 
with h5py.File('test.h5', 'w') as hdf:
    ds1 = hdf.create_dataset('arrA', data=A)
    ds2 = hdf.create_dataset('arrB', data=B)
    
    ds1.attrs['description'] = 'Здесь лежит массив А'
 
with h5py.File('test.h5', 'r') as hdf:
    ds1 = hdf['arrA']
    print(type(ds1))
    arr = ds1[:100]
# ..
 
ds1[:100]

<class 'h5py._hl.dataset.Dataset'>


ValueError: ignored

## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [49]:
import csv
import json


data_dict = {}

with open('/content/drive/MyDrive/Colab Notebooks/lab 5/tags_sample.csv') as file:
    reader = csv.reader(file)
    header = next(reader)
    print(header)
    for row in reader:
        
        item_id = int(row[0])
        item_tag = row[1]
        
        if item_id not in data_dict:
            data_dict[item_id] = []
        data_dict[item_id].append(item_tag)

['id', 'tag']


1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [ ]:
with open("/content/result.json", "w") as file:
    json.dump(data_dict, file)

with open("/content/result.json", "r") as file:
    data_dict_new = json.load(file)



import pandas as pd

recipes_filled = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lab 4/recipes_sample_with_filled_nsteps.csv")
#print(recipes_filled)


patched_dict = {k : len(v) for (k,v) in data_dict.items()}

def filler(x):
    if x["id"] in patched_dict:
        return patched_dict[x["id"]] 
    return float("nan")
    
recipes_filled['n_tags'] = recipes_filled.apply(lambda x: filler(x), axis=1)


print(recipes_filled)

patched_dict = {k : ";".join(v) for (k,v) in data_dict.items()}

print(patched_dict)



def filler(x):
    if x["id"] in patched_dict:
        return patched_dict[x["id"]]
    return float("nan")

recipes_filled['tags'] = recipes_filled.apply(lambda x: filler(x), axis=1)


print(recipes_filled)

print (recipes_filled[recipes_filled['n_tags'].isna()] )


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



           id  ...                                               tags
0       44123  ...  unsalted butter;carrot;onion;celery;broccoli s...
1       67664  ...         milk;frozen juice concentrate;plain yogurt
2       38798  ...  onion;frozen chopped spinach;eggs;garlic powde...
3       35173  ...  sandwich bun;good seasonings italian salad dre...
4       84797  ...  beef steaks;vegetable oil;spicy mustard;fresh ...
...       ...  ...                                                ...
29995  267661  ...  dry white wine;eggs;cheddar cheese;baking powd...
29996  386977  ...  unsalted butter;milk;flour;salt;vanilla;all-pu...
29997  103312  ...  onion;milk;eggs;butter;flour;salt;pepper;sugar...
29998  486161  ...  onion;celery;dried thyme;dried oregano;fresh p...
29999  298512  ...  butter;sour cream;egg;bisquick;light brown sug...

[30000 rows x 10 columns]
Empty DataFrame
Columns: [id, name, minutes, contributor_id, submitted, n_steps, description, n_ingredients, n_tags, tags]
Index: []


1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

In [ ]:
data_dict = {}

with open('/content/drive/MyDrive/Colab Notebooks/lab 5/ingredients_sample.csv') as file:
    reader = csv.DictReader(file)
    for item in reader:
        ingredient, recipe_id = item["ingredient"], int(item["recipe_id"])

        if recipe_id not in data_dict:
            data_dict[recipe_id] = []
        data_dict[recipe_id].append(ingredient)

        
data_dict



1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

In [51]:
import pandas as pd

patched_dict = {k : "*".join(v) for k, v in data_dict.items()}

def filler(x):
    if x["id"] in patched_dict:
        return patched_dict[x["id"]]
    return float("nan")
    
recipes_filled['ingredients'] = recipes_filled.apply(lambda x: filler(x), axis=1)


print (recipes_filled)


patched_dict = {k : len(v) for k, v in data_dict.items()}


def filler(x):
    if pd.isna(x["n_ingredients"]) and x["id"] in patched_dict:
        return patched_dict[x["id"]]
    return x["n_ingredients"]

recipes_filled['n_ingredients'] = recipes_filled.apply(lambda x: filler(x), axis=1)


print (recipes_filled)



           id  ...                                        ingredients
0       44123  ...  weeknight*time-to-make*course*main-ingredient*...
1       67664  ...  15-minutes-or-less*time-to-make*course*prepara...
2       38798  ...  30-minutes-or-less*time-to-make*course*main-in...
3       35173  ...  60-minutes-or-less*time-to-make*course*prepara...
4       84797  ...  30-minutes-or-less*time-to-make*course*main-in...
...       ...  ...                                                ...
29995  267661  ...  time-to-make*course*main-ingredient*cuisine*pr...
29996  386977  ...  time-to-make*course*main-ingredient*cuisine*pr...
29997  103312  ...  time-to-make*course*main-ingredient*cuisine*pr...
29998  486161  ...  ham*60-minutes-or-less*time-to-make*course*mai...
29999  298512  ...  30-minutes-or-less*time-to-make*course*prepara...

[30000 rows x 11 columns]
           id  ...                                        ingredients
0       44123  ...  weeknight*time-to-make*course*main-ingredie

1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

In [52]:
recipes_filled[recipes_filled["n_ingredients"].isnull()]


recipes_filled["n_ingredients"] = recipes_filled["n_ingredients"].astype(int)


print(recipes_filled.dtypes)

print (recipes_filled)

recipes_filled.to_csv("./recipes_sample_with_tags_ingredients.csv")

id                         int64
name                      object
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                    int64
description               object
n_ingredients              int64
n_tags                     int64
tags                      object
ingredients               object
dtype: object
           id  ...                                        ingredients
0       44123  ...  weeknight*time-to-make*course*main-ingredient*...
1       67664  ...  15-minutes-or-less*time-to-make*course*prepara...
2       38798  ...  30-minutes-or-less*time-to-make*course*main-in...
3       35173  ...  60-minutes-or-less*time-to-make*course*prepara...
4       84797  ...  30-minutes-or-less*time-to-make*course*main-in...
...       ...  ...                                                ...
29995  267661  ...  time-to-make*course*main-ingredient*cuisine*pr...
29996  386977  ...  time-to-make*course*main-ingredient*cuisine

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

In [53]:
import pandas as pd

recipes_filled["submitted"] = pd.to_datetime(recipes_filled["submitted"], format='%Y-%m-%d', errors='ignore')

print(recipes_filled)

print (recipes_filled.dtypes)



recipes_before = recipes_filled[recipes_filled["submitted"].dt.year < 2000]
print (recipes_before)

recipes_after = recipes_filled[recipes_filled["submitted"].dt.year >= 2000]
print (recipes_after)

print (recipes_after.dtypes)


recipes_after_np = np.array([recipes_after[item] for item in ["id", "minutes", "contributor_id","n_steps","n_ingredients","n_tags"]]) # recipes_after[["id", "minutes"]].values

print (recipes_after_np)

recipes_before_np = np.array([recipes_before[item] for item in ["id", "minutes", "contributor_id","n_steps","n_ingredients","n_tags"]])
print (recipes_before_np)



           id  ...                                        ingredients
0       44123  ...  weeknight*time-to-make*course*main-ingredient*...
1       67664  ...  15-minutes-or-less*time-to-make*course*prepara...
2       38798  ...  30-minutes-or-less*time-to-make*course*main-in...
3       35173  ...  60-minutes-or-less*time-to-make*course*prepara...
4       84797  ...  30-minutes-or-less*time-to-make*course*main-in...
...       ...  ...                                                ...
29995  267661  ...  time-to-make*course*main-ingredient*cuisine*pr...
29996  386977  ...  time-to-make*course*main-ingredient*cuisine*pr...
29997  103312  ...  time-to-make*course*main-ingredient*cuisine*pr...
29998  486161  ...  ham*60-minutes-or-less*time-to-make*course*mai...
29999  298512  ...  30-minutes-or-less*time-to-make*course*prepara...

[30000 rows x 11 columns]
id                         int64
name                      object
minutes                    int64
contributor_id             int64
s

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

In [ ]:
import h5py
import numpy as np

my_path = "./task_2_2.npz"
np.savez(my_path, recipes_before_2000=recipes_before_np, recipes_after_2000=recipes_after_np )

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

In [ ]:


data = np.load(my_path)
data.files

recipes_before_2000 = data["recipes_before_2000"]
print (recipes_before_2000)



recipes_after_2000 = data["recipes_after_2000"]
print(recipes_after_2000)



[[3.4410e+03 4.2050e+03 3.2580e+03 ... 3.7520e+03 4.8010e+03 2.9820e+03]
 [3.0000e+01 2.5000e+01 0.0000e+00 ... 0.0000e+00 2.0000e+01 0.0000e+00]
 [1.5620e+03 1.6170e+03 1.5340e+03 ... 1.5350e+03 1.5980e+03 1.2403e+05]
 [8.0000e+00 3.0000e+00 8.0000e+00 ... 1.3000e+01 4.0000e+00 6.0000e+00]
 [8.0000e+00 5.0000e+00 6.0000e+00 ... 4.0000e+00 7.0000e+00        nan]
 [8.0000e+00 5.0000e+00 6.0000e+00 ... 4.0000e+00 7.0000e+00 7.0000e+00]]
[[4.41230e+04 6.76640e+04 3.87980e+04 ... 1.03312e+05 4.86161e+05
  2.98512e+05]
 [9.00000e+01 1.00000e+01 3.00000e+01 ... 7.50000e+01 6.00000e+01
  2.90000e+01]
 [3.51930e+04 9.19700e+04 1.53300e+03 ... 1.61745e+05 2.27978e+05
  5.06822e+05]
 [1.10000e+01 3.00000e+00 5.00000e+00 ... 1.00000e+01 7.00000e+00
  9.00000e+00]
 [1.80000e+01         nan 8.00000e+00 ...         nan         nan
  1.00000e+01]
 [1.80000e+01 3.00000e+00 8.00000e+00 ... 1.30000e+01 2.20000e+01
  1.00000e+01]]


### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

In [ ]:
datasets_dict = {}
with h5py.File("/content/drive/MyDrive/Colab Notebooks/lab 5/nutrition_sample.h5","r") as hdf:
    
    for key in hdf.keys():
        file = hdf[key]
        metadata = dict(file.attrs)
        print(f"Dataset name={key}, dataset size={file.shape}, metadata={metadata}")
        datasets_dict[key] = {"all" : {'data': file[:]}, 'attrs': metadata}



datasets_dict



Dataset name=dataset_0, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}
Dataset name=dataset_2, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}
Dataset name=dataset_3, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sodium (PDV)'}
Dataset name=dataset_4, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'protein (PDV)'}
Dataset name=dataset_5, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'saturated fat (PDV)'}
Dataset name=dataset_6, dataset size=(30000, 2), metadata={'col_0': 'recipe_id', 'col_1': 'carbohydrates (PDV)'}


{'dataset_0': {'all': {'data': array([[4.41230e+04, 8.04700e+02],
          [6.76640e+04, 1.64600e+02],
          [3.87980e+04, 5.38000e+01],
          ...,
          [1.03312e+05, 8.64100e+02],
          [4.86161e+05, 4.15200e+02],
          [2.98512e+05, 1.88000e+02]])},
  'attrs': {'col_0': 'recipe_id', 'col_1': 'calories (#)'}},
 'dataset_1': {'all': {'data': array([[4.41230e+04, 1.08000e+02],
          [6.76640e+04, 3.00000e+00],
          [3.87980e+04, 5.00000e+00],
          ...,
          [1.03312e+05, 8.70000e+01],
          [4.86161e+05, 2.60000e+01],
          [2.98512e+05, 1.10000e+01]])},
  'attrs': {'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}},
 'dataset_2': {'all': {'data': array([[4.41230e+04, 2.60000e+01],
          [6.76640e+04, 5.00000e+00],
          [3.87980e+04, 2.00000e+00],
          ...,
          [1.03312e+05, 3.00000e+01],
          [4.86161e+05, 3.40000e+01],
          [2.98512e+05, 5.70000e+01]])},
  'attrs': {'col_0': 'recipe_id', 'col_1': 'sugar (PD

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

In [ ]:
datasets_dict = {k : v for k, v in datasets_dict.items() if "PDV" in v["attrs"]["col_1"]}

new_datasets_dict = {"PDV_more_100" : {}, "PDV_less_100": {}}
for dataset, values in datasets_dict.items():

    data = values["all"]["data"]
    mask = data[:,1] > 1
    
    new_datasets_dict["PDV_more_100"][dataset] = {"data" :  data[mask], "attrs" : values["attrs"], "name": dataset}

    mask = data[:,1] < 1
    new_datasets_dict["PDV_less_100"][dataset] = {"data" :  data[mask], "attrs" : values["attrs"], "name": dataset}


for dataset, data in new_datasets_dict.items():
    print(dataset)
    print(data)




h5_path = "./nutrition_grouped.h5"
with h5py.File(h5_path, mode='w') as hdf:
    for group, data in new_datasets_dict.items():
        new_group = hdf.create_group(group)
        for d in data:
            item = new_group.create_dataset(name=data[d]['name'], data=data[d]['data'])
            item.attrs.update(data[d]['attrs'])



PDV_more_100
{'dataset_1': {'data': array([[4.41230e+04, 1.08000e+02],
       [6.76640e+04, 3.00000e+00],
       [3.87980e+04, 5.00000e+00],
       ...,
       [1.03312e+05, 8.70000e+01],
       [4.86161e+05, 2.60000e+01],
       [2.98512e+05, 1.10000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}, 'name': 'dataset_1'}, 'dataset_2': {'data': array([[4.41230e+04, 2.60000e+01],
       [6.76640e+04, 5.00000e+00],
       [3.87980e+04, 2.00000e+00],
       ...,
       [1.03312e+05, 3.00000e+01],
       [4.86161e+05, 3.40000e+01],
       [2.98512e+05, 5.70000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}, 'name': 'dataset_2'}, 'dataset_3': {'data': array([[4.41230e+04, 1.90000e+01],
       [3.87980e+04, 3.00000e+00],
       [3.51730e+04, 9.70000e+01],
       ...,
       [1.03312e+05, 1.80000e+01],
       [4.86161e+05, 2.60000e+01],
       [2.98512e+05, 1.10000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'sodium (PDV)'}, 'name': 'dataset_3'}, 'datas

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

In [ ]:
with h5py.File(h5_path) as hdf:
    
    for group_name, data in hdf.items():
        print(f'\nGroup: {group_name}')

        for key in data:
            file = data[key]
            metadata = dict(file.attrs)
            print(f"Dataset name={key}, dataset size={file.shape}, metadata={metadata}")
            datasets_dict[key] = {"all" : {'data': file[:]}, 'attrs': metadata}


Group: PDV_less_100
Dataset name=dataset_1, dataset size=(2146, 2), metadata={'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}
Dataset name=dataset_2, dataset size=(1371, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}
Dataset name=dataset_3, dataset size=(2588, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sodium (PDV)'}
Dataset name=dataset_4, dataset size=(1274, 2), metadata={'col_0': 'recipe_id', 'col_1': 'protein (PDV)'}
Dataset name=dataset_5, dataset size=(2569, 2), metadata={'col_0': 'recipe_id', 'col_1': 'saturated fat (PDV)'}
Dataset name=dataset_6, dataset size=(1653, 2), metadata={'col_0': 'recipe_id', 'col_1': 'carbohydrates (PDV)'}

Group: PDV_more_100
Dataset name=dataset_1, dataset size=(27025, 2), metadata={'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}
Dataset name=dataset_2, dataset size=(27822, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}
Dataset name=dataset_3, dataset size=(26148, 2), metadata={'col_0': 'recipe_id', 'col_1': 'sodium

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.

In [ ]:
h5_compressed_path = "./nutrition_grouped_compressed.h5"
with h5py.File(h5_compressed_path, mode='w') as hdf:
    for group, data in new_datasets_dict.items():
        new_group = hdf.create_group(group)
        for d in data:
            item = new_group.create_dataset(name=data[d]['name'], data=data[d]['data'], compression="gzip", compression_opts=9)
            item.attrs.update(data[d]['attrs'])


def get_file_size_2(file):
 
    stat = os.stat(file)
    print(stat)
    size = stat.st_size
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    elif unit == "GB":
        return print('File size: ' + str(round(size / (1024 * 1024 * 1024), 3)) + ' Gigabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')


import os


size = get_file_size_2(h5_path)
print('File size: ' + str(size) + ' bytes')

size = get_file_size_2(h5_compressed_path)
print('File size: ' + str(size) + ' bytes')

os.stat_result(st_mode=33188, st_ino=3801228, st_dev=51, st_nlink=1, st_uid=0, st_gid=0, st_size=2788576, st_atime=1633959601, st_mtime=1633959590, st_ctime=1633959590)
File size: 2788576 bytes
os.stat_result(st_mode=33188, st_ino=3801229, st_dev=51, st_nlink=1, st_uid=0, st_gid=0, st_size=914789, st_atime=1633959652, st_mtime=1633959653, st_ctime=1633959653)
File size: 914789 bytes
